# Annotate Mitosis Events
can be applied to other cell events  
Author: Ke



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
# from livecellx import segment
from livecellx import core
from livecellx.core import datasets
from livecellx.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecellx.core import SingleCellTrajectory, SingleCellStatic
# import detectron2
# from detectron2.utils.logger import setup_logger

# setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import cv2

# import some common detectron2 utilities
# from detectron2 import model_zoo
# from detectron2.engine import DefaultPredictor
# from detectron2.config import get_cfg
# from detectron2.utils.visualizer import Visualizer
# from detectron2.data import MetadataCatalog, DatasetCatalog
# from livecellx.segment.detectron_utils import gen_cfg

# from livecellx.segment.detectron_utils import (
#     segment_detectron_wrapper,
#     segment_images_by_detectron,
#     convert_detectron_instance_pred_masks_to_binary_masks,
#     convert_detectron_instances_to_label_masks,
# )
# from livecellx.segment.detectron_utils import (
#     convert_detectron_instance_pred_masks_to_binary_masks,
#     convert_detectron_instances_to_label_masks,
#     segment_images_by_detectron,
#     segment_single_img_by_detectron_wrapper,
# )


## Loading single cells from existing mask files

```LiveCellImageDataset, SingleCellImageDataset``` from livecellx.core.datasets allow users to load images with ease, without reading directly into memories.  
In `mask_dataset_path` please make sure that the sorted (alphabetically) file names correspond to the order of times.
 Note that the sorted mechanism provided is simply sort the url (file name) list according to string value. Please note that without proper left trailing zeroes, the order of final times may be incorrect. E.g. string  `T10` (10th file) is less than string `T2`. If you have your customized file patterns, please provide `LiveCellImageDataset` with a `time2url` dictionary to provide necessary time information mapped to file locations for reading time-lapsed data.
`SingleCellImageDataset` takes a single image from the memory and makes it a single time point dataset, which can be handy when you would like to process imaging datasets. 

In [ ]:
# dataset_dir_path = Path(
#     "../datasets/test_data_STAV-A549/DIC_data"
# )

# mask_dataset_path = Path("../datasets/test_data_STAV-A549/mask_data")

# dataset_dir_path = Path(
#     "../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/XY16/"
# )

# mask_dataset_path = Path("../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/out/XY16/seg")
raw_image_dir_path = Path(
    "../datasets/tutorial_data_EBSS_mitosis/raw_images"
)

mask_dataset_path = Path("../datasets/tutorial_data_EBSS_mitosis/label_masks")


In [ ]:
mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
len(mask_dataset)

In [ ]:
time2url = sorted(glob.glob(str((Path(raw_image_dir_path) / Path("*_DIC.tif")))))
time2url = {i: path for i, path in enumerate(time2url)}
dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")
# dic_dataset = LiveCellImageDataset(dataset_dir_path, ext="tif")

In [ ]:
len(dic_dataset)

### Convert label masks to single objects

In [ ]:
from skimage.measure import regionprops
from livecellx.segment.utils import prep_scs_from_mask_dataset
single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

In [ ]:
single_cells[0].mask_dataset.name, single_cells[1].mask_dataset.name

In [ ]:
single_cells_by_time = {}
for cell in single_cells:
    if cell.timeframe not in single_cells_by_time:
        single_cells_by_time[cell.timeframe] = []
    single_cells_by_time[cell.timeframe].append(cell)

## Correct Single Cell Segmentation before Tracking

In [ ]:
from livecellx.core.single_cell import SingleCellTrajectoryCollection
temp_sc_trajs_for_correct = SingleCellTrajectoryCollection()
for idx, sc in enumerate(single_cells):
    sct = SingleCellTrajectory(track_id = idx, timeframe_to_single_cell={sc.timeframe: sc})
    temp_sc_trajs_for_correct.add_trajectory(sct)


In [ ]:
# # load csn model?
from livecellx.core.sc_seg_operator import ScSegOperator
ckpt_path = r"./notebook_results/csn_models/model_v11_epoch=3282-test_loss=2.3688.ckpt"
ScSegOperator.load_default_csn_model(path=ckpt_path, cuda=True);

## Tracking based on single cells

In [ ]:
from typing import List
from livecellx.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs
)


traj_collection = track_SORT_bbox_from_scs(single_cells, dic_dataset, mask_dataset=mask_dataset, max_age=1, min_hits=1)

In [ ]:
traj_collection.histogram_traj_length()

In [ ]:
%gui qt
from livecellx.core.napari_visualizer import NapariVisualizer
import napari
from skimage import data


In [ ]:
from livecellx.core.single_cell import SingleCellStatic, SingleCellTrajectory, SingleCellTrajectoryCollection
import numpy as np
from napari.viewer import Viewer
from livecellx.core.visualizer import Visualizer

from livecellx.core.single_cell import SingleCellStatic, SingleCellTrajectory, SingleCellTrajectoryCollection
import numpy as np
from napari.viewer import Viewer
from livecellx.core.visualizer import Visualizer

### Manually or programmatically correct trajectories via SctOperator

In [ ]:
import napari
from livecellx.core.sct_operator import SctOperator, create_sct_napari_ui
viewer = napari.view_image(dic_dataset.to_dask(), name="dic_image", cache=True)
shape_layer = NapariVisualizer.gen_trajectories_shapes(traj_collection, viewer, contour_sample_num=20)
shape_layer.mode = "select"

sct_operator = SctOperator(traj_collection, shape_layer, viewer)
# sct_operator.setup_shape_layer(shape_layer)
create_sct_napari_ui(sct_operator)

Now please annotate some cells in the napari UI by choosing "click&annotate"

Save programmatically

In [ ]:
sample_out_dir = Path("./test_sample_out")
sample_dataset_dir = sample_out_dir / "datasets"
filename_pattern = "sample_{sample_index}.json"
sample_paths = sct_operator.save_annotations(sample_out_dir / "mitosis_samples", filename_pattern, sample_dataset_dir=sample_dataset_dir)

### Check if samples are saved successfully

In [ ]:
# Load and Check if samples are saved successfully
sample_path = sample_paths[0]
sample = SingleCellStatic.load_single_cells_json(sample_path, dataset_json_dir = sample_dataset_dir)
print(sample)

In [ ]:
SingleCellStatic.write_single_cells_json(single_cells, sample_out_dir / "all_cells.json" ,dataset_dir = sample_dataset_dir)